In [20]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [21]:
data=pd.read_csv("../merged/len4_ndb.csv",encoding="shift-jis",index_col=0)
# drop never treated group
data=data.dropna(subset=["Approved"])
data.head()

C:\Users\hakos\AppData\Local\Temp\ipykernel_8520\298005430.py:1: DtypeWarning: Columns (14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("../merged/len4_ndb.csv",encoding="shift-jis",index_col=0)


,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,oral,id_l4,Approved,KEGG,Ingredient,variable,id,revenue
0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,292190.95,2014,1,0,1145,1985.0,D00141,Indometacin,id1,1145.0,5.639285e+06
1,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,128832.00,2014,1,0,1145,1985.0,D00141,Indometacin,id1,1145.0,2.486458e+06
2,114.0,解熱鎮痛消炎剤,661140020.0,インドメタシン坐剤 ５０ｍｇ,1145700J3012,19.3,0.0,106328.00,2014,1,0,1145,1985.0,D00141,Indometacin,id1,1145.0,2.052130e+06
3,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,658257.00,2014,0,0,1145,1985.0,D00141,Indometacin,id1,1145.0,1.270436e+07
4,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,418727.50,2014,0,0,1145,1985.0,D00141,Indometacin,id1,1145.0,8.081441e+06


1 observation が複数行にまたがっているので経過年数ダミー以外は落として1行にまとめよう

In [22]:
data["elasped"]=data["year"]-data["Approved"]
#　不要なところを除外
# data.loc[(data['elasped'] > 10) | (data["elasped"]< -5), 'elasped'] = np.nan

# dummy をとりたいところを分離
dummies=pd.get_dummies(data["elasped"],prefix="elasped")
dummies.columns = dummies.columns.str.replace('-', 'm').str.replace(r'\..*', '', regex=True)
# sumをとりたいところを分離
sumcolumns=["総計","revenue"]
sums=data.groupby(["year","id","後発品区分","in_hospital","oral"])[sumcolumns].sum().reset_index()
# save temporarily
colnames=dummies.columns.to_list()
# count ver.
# aggregated_data = pd.concat([data.groupby(['year', 'id']).first(), dummies.groupby([data['year'], data['id']]).sum()], axis=1).reset_index()

# dummy ver.
# aggregated_df = pd.concat([
#     data.groupby(['year', 'id',"後発品区分","in_hospital","oral"]).first(), 
#     dummies.groupby([data['year'], data['id'],data["後発品区分"],data["in_hospital"],data["oral"]]).apply(lambda x: (x[colnames] > 0).any().astype(int))
# ], axis=1).reset_index()
# # sumをとったところを追加する
# aggregated_df[sumcolumns]=sums[sumcolumns]
# aggregated_df
# aggregated_df.head()

In [23]:
sums.rename(columns={"総計":"sum_quantity","revenue":"sum_revenue"},inplace=True)

In [24]:
merged_data=pd.merge(data,sums,on=["year","id","後発品区分","in_hospital","oral"],how="left")

In [25]:
merged_data.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,...,id_l4,Approved,KEGG,Ingredient,variable,id,revenue,elasped,sum_quantity,sum_revenue
0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,292190.95,2014,1,...,1145,1985.0,D00141,Indometacin,id1,1145.0,5.639285e+06,29.0,527350.95,1.017787e+07
1,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,128832.00,2014,1,...,1145,1985.0,D00141,Indometacin,id1,1145.0,2.486458e+06,29.0,527350.95,1.017787e+07
2,114.0,解熱鎮痛消炎剤,661140020.0,インドメタシン坐剤 ５０ｍｇ,1145700J3012,19.3,0.0,106328.00,2014,1,...,1145,1985.0,D00141,Indometacin,id1,1145.0,2.052130e+06,29.0,527350.95,1.017787e+07
3,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,658257.00,2014,0,...,1145,1985.0,D00141,Indometacin,id1,1145.0,1.270436e+07,29.0,1076984.50,2.078580e+07
4,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,418727.50,2014,0,...,1145,1985.0,D00141,Indometacin,id1,1145.0,8.081441e+06,29.0,1076984.50,2.078580e+07


connect generic usage

In [26]:
dusage=pd.read_csv("../generic/generic_usage_imp.csv")
dusage

,Year,Percentage
0,2007,34.90
1,2008,35.35
2,2009,35.80
3,2010,37.85
4,2011,39.90
5,2012,43.40
6,2013,46.90
7,2014,51.55
8,2015,56.20
9,2016,61.00


In [27]:
merged_df=merged_data.merge(dusage,left_on="year",right_on="Year")
merged_df.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,...,KEGG,Ingredient,variable,id,revenue,elasped,sum_quantity,sum_revenue,Year,Percentage
0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,292190.95,2014,1,...,D00141,Indometacin,id1,1145.0,5.639285e+06,29.0,527350.95,1.017787e+07,2014,51.55
1,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,128832.00,2014,1,...,D00141,Indometacin,id1,1145.0,2.486458e+06,29.0,527350.95,1.017787e+07,2014,51.55
2,114.0,解熱鎮痛消炎剤,661140020.0,インドメタシン坐剤 ５０ｍｇ,1145700J3012,19.3,0.0,106328.00,2014,1,...,D00141,Indometacin,id1,1145.0,2.052130e+06,29.0,527350.95,1.017787e+07,2014,51.55
3,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,658257.00,2014,0,...,D00141,Indometacin,id1,1145.0,1.270436e+07,29.0,1076984.50,2.078580e+07,2014,51.55
4,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,418727.50,2014,0,...,D00141,Indometacin,id1,1145.0,8.081441e+06,29.0,1076984.50,2.078580e+07,2014,51.55


In [28]:
id_dummies=pd.get_dummies(merged_df["id"],prefix="id")
df_with_dummies=pd.concat([merged_df,id_dummies],axis=1)
df_with_dummies.rename(columns={"Percentage":"generic_per"},inplace=True)
df_with_dummies.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,...,id_3327.0,id_3399.0,id_3919.0,id_3992.0,id_3999.0,id_4413.0,id_4490.0,id_6250.0,id_6290.0,id_7990.0
0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,292190.95,2014,1,...,0,0,0,0,0,0,0,0,0,0
1,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,128832.00,2014,1,...,0,0,0,0,0,0,0,0,0,0
2,114.0,解熱鎮痛消炎剤,661140020.0,インドメタシン坐剤 ５０ｍｇ,1145700J3012,19.3,0.0,106328.00,2014,1,...,0,0,0,0,0,0,0,0,0,0
3,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,658257.00,2014,0,...,0,0,0,0,0,0,0,0,0,0
4,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,418727.50,2014,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
# df_with_dummies.to_csv("len3_agg.csv",encoding="shift-jis")
df_with_dummies.to_csv("../merged/len4_ndb_agg_blp_DropNever.csv",encoding="shift-jis")